## Random forest for SRQ prediction with more feature engineering

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import sklearn
import warnings
import joblib
import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import f1_score, mean_squared_error, r2_score, mean_absolute_error, accuracy_score
from sklearn.ensemble import RandomForestRegressor

warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('Datasets/FlightsByDay-SRQ-2025_07_21_09_49_46.csv')
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df.info()

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(df['Date'], df['Boarded'], marker='o', linestyle='-')
plt.title('Passenger Count Over Time')
plt.xlabel('Date')
plt.ylabel('Passenger Count')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Extract time-based features
#df['Hour'] = df['Flight Time'].dt.hour
df['Day_of_Week'] = df['Date'].dt.dayofweek  # 0=Monday, 6=Sunday
df['Month'] = df['Date'].dt.month
df['Day_of_Month'] = df['Date'].dt.day

# Create time-based categories
#df['Time_Category'] = pd.cut(df['Hour'], 
                           #bins=[0, 6, 12, 18, 24], 
                           #labels=['Night', 'Morning', 'Afternoon', 'Evening'])

# Encode categorical variables
label_encoders = {}
categorical_columns = ['Destination Airport', 'Airline']

for col in categorical_columns:
    le = LabelEncoder()
    df[col + '_encoded'] = le.fit_transform(df[col])
    label_encoders[col] = le


df

## Cyclical encoding of dates

In [ ]:
df['day_sin'] = np.sin(2*np.pi*df['Day_of_Month']/31)
df['day_cos'] = np.cos(2*np.pi*df['Day_of_Month']/31)

# Month (1–12) → convert to 0–11 by subtracting 1, then encode
df['month_sin'] = np.sin(2 * np.pi * (df['Month'] - 1) / 12)
df['month_cos'] = np.cos(2 * np.pi * (df['Month'] - 1) / 12)

# Day of Week → if 0=Monday…6=Sunday, just divide by 7
df['dow_sin'] = np.sin(2 * np.pi * df['Day_of_Week'] / 7)
df['dow_cos'] = np.cos(2 * np.pi * df['Day_of_Week'] / 7)

df.info()

## group by airline and destination

In [ ]:
route_stats = df.groupby(['Airline','Destination Airport'])['Boarded'].agg([
    ('route_mean','mean'),
    ('route_median','median'),
    ('route_std','std'),
]).reset_index()
df = df.merge(route_stats, on=['Airline','Destination Airport'], how='left')
df

In [ ]:
# Select features for modeling
feature_columns = [
    'Destination Airport_encoded', 
    'Airline_encoded', 'Day_of_Week', 'Month', 
    'Day_of_Month', 'day_sin', 'day_cos', 'month_sin',
    'month_cos', 'dow_sin', 'dow_cos', 'route_mean',
    'route_median', 'route_std'
]

X = df[feature_columns]
y = df['Boarded']

print("Feature matrix shape:", X.shape)
print("Target shape:", y.shape)

In [ ]:
#Need to drop time stamps for modeling but need the timestamps to present data
df_timestamps = df['Date']

split_factor = 0.8
split_index = int(len(df) * split_factor)

train_df = df[:split_index]
test_df = df[split_index:]

# Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, stratify=None
# )

# Extract X and y from train and test sets
X_train = train_df[feature_columns]
y_train = train_df['Boarded']

X_test = test_df[feature_columns]
y_test = test_df['Boarded']


print(f"Training set size: {X_train.shape[0]}")
print(f"Testing set size: {X_test.shape[0]}")
print(f"Training target range: {y_train.min()} to {y_train.max()}")
print(f"Testing target range: {y_test.min()} to {y_test.max()}")

In [ ]:
plt.figure(figsize=(20,8))

plt.plot(df_timestamps[:split_index], y_train, label='Train', color='blue')
plt.plot(df_timestamps[split_index:], y_test, label='Test', color='orange')

plt.title('Passenger Count Over Time (Train/Test Split)')
plt.xlabel('Flight Time')
plt.ylabel('Passenger Count')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
param_grid_random = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [10, 20, 30, 40, 50, None],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}

In [ ]:
rf = RandomForestRegressor()
tscv= TimeSeriesSplit(3)

In [ ]:
print("Starting Randomized Search...")
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_grid_random,
    n_iter=100,  # Number of parameter combinations to try
    cv=tscv,
    scoring='neg_mean_squared_error',  # Use 'neg_mean_squared_error' for regression
    n_jobs=-1,
    verbose=1,
    random_state=42
)

In [ ]:
random_search.fit(X_train, y_train)

In [ ]:
# Get the best model
best_rf = random_search.best_estimator_
print(f"Best parameters: {random_search.best_params_}")
print(f"Best cross-validation score: {random_search.best_score_:.4f}")

# Make predictions
y_pred = best_rf.predict(X_test)

In [ ]:
def predict_passengers(model, flight_time, dest_airport, airline,
                       label_encoders, route_stats, feature_columns):
    # 1) Build base DataFrame
    sample = pd.DataFrame({'Flight Time': [flight_time]})
    
    # 2) Extract raw time features
    dt = pd.to_datetime(sample['Flight Time'])
    sample['Day_of_Month']  = dt.dt.day
    sample['Month']         = dt.dt.month
    sample['Day_of_Week']   = dt.dt.dayofweek  # 0=Mon … 6=Sun
    
    # 3) Cyclical encodings
    sample['day_sin']   = np.sin(2 * np.pi * sample['Day_of_Month'] / 31)
    sample['day_cos']   = np.cos(2 * np.pi * sample['Day_of_Month'] / 31)
    sample['month_sin'] = np.sin(2 * np.pi * (sample['Month'] - 1) / 12)
    sample['month_cos'] = np.cos(2 * np.pi * (sample['Month'] - 1) / 12)
    sample['dow_sin']   = np.sin(2 * np.pi * sample['Day_of_Week'] / 7)
    sample['dow_cos']   = np.cos(2 * np.pi * sample['Day_of_Week'] / 7)
    
    # 4) Categorical encodings
    try:
        sample['Destination Airport_encoded'] = label_encoders['Destination Airport']\
            .transform([dest_airport])[0]
        sample['Airline_encoded'] = label_encoders['Airline']\
            .transform([airline])[0]
    except KeyError as e:
        return f"Error: Missing encoder for {e}"
    except ValueError as e:
        return f"Error: Unknown category - {e}"
    
    # 5) Lookup route statistics
    #    We assume route_stats index on ['Airline','Destination Airport']
    key = (airline, dest_airport)
    try:
        stats = route_stats.set_index(['Airline','Destination Airport'])\
                          .loc[key, ['route_mean','route_median','route_std']]
        sample['route_mean']   = stats['route_mean']
        sample['route_median'] = stats['route_median']
        sample['route_std']    = stats['route_std']
    except KeyError:
        # fallback if this route never seen in training
        sample['route_mean']   = route_stats['route_mean'].mean()
        sample['route_median'] = route_stats['route_median'].median()
        sample['route_std']    = route_stats['route_std'].mean()
    
    # 6) Select features and predict
    try:
        X_sample = sample[feature_columns]
        pred = model.predict(X_sample)[0]
        return int(round(pred))
    except Exception as e:
        return f"Error during prediction: {e}"


In [ ]:
# Example usage
example_prediction = predict_passengers(
    best_rf,
    flight_time="2025-07-22 14:38:00",
    dest_airport="DFW",
    airline="AA",
    label_encoders=label_encoders,
    route_stats=route_stats,
    feature_columns=feature_columns
)
    
print(f"Predicted passenger count: {example_prediction}")

In [ ]:
# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Model Performance on Test Set:")
print(f"Mean Squared Error: {mse:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"R-squared Score: {r2:.4f}")
#print(f"Out-of-Bag Score: {best_rf.oob_score_:.4f}")

# Context for interpretation
print(f"\nFor context:")
print(f"Average passenger count: {y_test.mean():.1f}")
print(f"Typical prediction error: ±{rmse:.1f} passengers")

In [ ]:
# Plot actual vs predicted
plt.figure(figsize=(12, 5))

# Line plot
plt.subplot(1, 2, 1)
plt.plot(y_test.values[:100], label='Actual', marker='o', markersize=4)
plt.plot(y_pred[:100], label='Predicted', marker='s', markersize=4)
plt.title('Actual vs Predicted (First 100 samples)')
plt.xlabel('Sample Index')
plt.ylabel('Passenger Count')
plt.legend()
plt.grid(True, alpha=0.3)

# Scatter plot
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Passenger Count')
plt.ylabel('Predicted Passenger Count')
plt.title('Actual vs Predicted (Scatter)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Get feature importance
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': best_rf.feature_importances_
}).sort_values('importance', ascending=False)

print("Feature Importance:")
print(feature_importance)

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['feature'], feature_importance['importance'])
plt.xlabel('Importance')
plt.title('Feature Importance in Random Forest Model')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()